# Analisis dan Prediksi Harga Kos-Kosan di Jakarta

## Ringkasan Project

Project ini menganalisis faktor-faktor yang mempengaruhi harga kos-kosan di Jakarta dan membangun model machine learning untuk memprediksi harga berdasarkan lokasi dan fasilitas yang tersedia.

**Tujuan:**
- Mengidentifikasi faktor utama yang mempengaruhi harga kos
- Membangun model prediksi harga yang akurat
- Memberikan insight untuk pencari kos dan pemilik kos

**Dataset:** 900+ data kos dari berbagai wilayah Jakarta

**Author:** Data Science Portfolio Project  
**Tanggal:** November 2024

---

## Daftar Isi

1. [Import Libraries](#import)
2. [Load dan Eksplorasi Data](#load-data)
3. [Analisis Deskriptif](#analisis-deskriptif)
4. [Analisis Berdasarkan Lokasi](#analisis-lokasi)
5. [Analisis Fasilitas](#analisis-fasilitas)
6. [Feature Engineering](#feature-engineering)
7. [Model Building](#model-building)
8. [Evaluasi Model](#evaluasi-model)
9. [Feature Importance](#feature-importance)
10. [Kesimpulan dan Rekomendasi](#kesimpulan)

---

<a id='import'></a>
## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("Libraries berhasil di-import")

<a id='load-data'></a>
## 2. Load dan Eksplorasi Data

In [ ]:
# Load dataset
df = pd.read_csv('data_kos_jakarta.csv')

print("Dimensi dataset:", df.shape)
print("\n" + "="*80)
print("Sample Data:")
df.head(10)

In [ ]:
# Info dataset
print("Informasi Dataset:")
print("="*80)
df.info()

In [ ]:
# Statistik deskriptif
print("Statistik Deskriptif:")
print("="*80)
df.describe()

In [ ]:
# Cek missing values
print("Missing Values:")
print("="*80)
missing = df.isnull().sum()
if missing.sum() == 0:
    print("Tidak ada missing values")
else:
    print(missing[missing > 0])

<a id='analisis-deskriptif'></a>
## 3. Analisis Deskriptif

### 3.1 Distribusi Harga Kos

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(df['harga_per_bulan'], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Harga per Bulan (Rp)', fontsize=12)
axes[0].set_ylabel('Frekuensi', fontsize=12)
axes[0].set_title('Distribusi Harga Kos di Jakarta', fontsize=14, fontweight='bold')
axes[0].axvline(df['harga_per_bulan'].median(), color='red', linestyle='--', 
                label=f'Median: Rp {df["harga_per_bulan"].median():,.0f}')
axes[0].legend()

# Box plot
axes[1].boxplot(df['harga_per_bulan'], vert=True)
axes[1].set_ylabel('Harga per Bulan (Rp)', fontsize=12)
axes[1].set_title('Box Plot Harga Kos', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Harga Terendah: Rp {df['harga_per_bulan'].min():,.0f}")
print(f"Harga Tertinggi: Rp {df['harga_per_bulan'].max():,.0f}")
print(f"Harga Rata-rata: Rp {df['harga_per_bulan'].mean():,.0f}")
print(f"Harga Median: Rp {df['harga_per_bulan'].median():,.0f}")

### 3.2 Distribusi Tipe Kos

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
tipe_counts = df['tipe_kos'].value_counts()
axes[0].bar(tipe_counts.index, tipe_counts.values, color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_xlabel('Tipe Kos', fontsize=12)
axes[0].set_ylabel('Jumlah', fontsize=12)
axes[0].set_title('Distribusi Tipe Kos', fontsize=14, fontweight='bold')
for i, v in enumerate(tipe_counts.values):
    axes[0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Pie chart
axes[1].pie(tipe_counts.values, labels=tipe_counts.index, autopct='%1.1f%%',
           colors=['#3498db', '#e74c3c', '#2ecc71'], startangle=90)
axes[1].set_title('Proporsi Tipe Kos', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

<a id='analisis-lokasi'></a>
## 4. Analisis Berdasarkan Lokasi

### 4.1 Harga Rata-rata per Kota

In [ ]:
# Harga rata-rata per kota
harga_per_kota = df.groupby('kota')['harga_per_bulan'].agg(['mean', 'median', 'min', 'max', 'count'])
harga_per_kota = harga_per_kota.sort_values('mean', ascending=False)

print("Statistik Harga per Kota:")
print("="*80)
print(harga_per_kota)

# Visualisasi
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Bar plot rata-rata
harga_per_kota['mean'].plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Harga Rata-rata (Rp)', fontsize=12)
axes[0].set_ylabel('Kota', fontsize=12)
axes[0].set_title('Harga Rata-rata Kos per Kota di Jakarta', fontsize=14, fontweight='bold')
for i, v in enumerate(harga_per_kota['mean']):
    axes[0].text(v + 30000, i, f'Rp {v:,.0f}', va='center')

# Box plot
df.boxplot(column='harga_per_bulan', by='kota', ax=axes[1])
axes[1].set_xlabel('Kota', fontsize=12)
axes[1].set_ylabel('Harga per Bulan (Rp)', fontsize=12)
axes[1].set_title('Distribusi Harga per Kota', fontsize=14, fontweight='bold')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

### 4.2 Pengaruh Jarak ke Kampus terhadap Harga

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(df['jarak_ke_kampus_km'], df['harga_per_bulan'], alpha=0.5, color='steelblue')
axes[0].set_xlabel('Jarak ke Kampus (km)', fontsize=12)
axes[0].set_ylabel('Harga per Bulan (Rp)', fontsize=12)
axes[0].set_title('Hubungan Jarak ke Kampus dengan Harga', fontsize=14, fontweight='bold')

# Binned analysis
df['jarak_kategori'] = pd.cut(df['jarak_ke_kampus_km'], bins=[0, 2, 5, 10, 15], 
                               labels=['<2 km', '2-5 km', '5-10 km', '>10 km'])
harga_by_jarak = df.groupby('jarak_kategori')['harga_per_bulan'].mean()
axes[1].bar(range(len(harga_by_jarak)), harga_by_jarak.values, color='coral')
axes[1].set_xticks(range(len(harga_by_jarak)))
axes[1].set_xticklabels(harga_by_jarak.index, rotation=0)
axes[1].set_xlabel('Kategori Jarak', fontsize=12)
axes[1].set_ylabel('Harga Rata-rata (Rp)', fontsize=12)
axes[1].set_title('Harga Rata-rata Berdasarkan Jarak ke Kampus', fontsize=14, fontweight='bold')
for i, v in enumerate(harga_by_jarak.values):
    axes[1].text(i, v + 30000, f'Rp {v:,.0f}', ha='center')

plt.tight_layout()
plt.show()

print("\nHarga Rata-rata Berdasarkan Jarak ke Kampus:")
print("="*80)
print(harga_by_jarak)

<a id='analisis-fasilitas'></a>
## 5. Analisis Fasilitas

### 5.1 Perbandingan Harga: Ada vs Tidak Ada Fasilitas

In [ ]:
# Fasilitas yang akan dianalisis
fasilitas_cols = ['ac', 'kamar_mandi_dalam', 'wifi', 'listrik_include', 
                  'parkir', 'dapur', 'laundry', 'security_24jam']
fasilitas_labels = ['AC', 'Kamar Mandi Dalam', 'WiFi', 'Listrik Include',
                    'Parkir', 'Dapur', 'Laundry', 'Security 24 Jam']

# Analisis perbedaan harga
hasil_analisis = []
for col, label in zip(fasilitas_cols, fasilitas_labels):
    harga_ada = df[df[col] == 1]['harga_per_bulan'].mean()
    harga_tidak_ada = df[df[col] == 0]['harga_per_bulan'].mean()
    selisih = harga_ada - harga_tidak_ada
    persentase = (selisih / harga_tidak_ada) * 100
    
    hasil_analisis.append({
        'Fasilitas': label,
        'Harga (Ada)': harga_ada,
        'Harga (Tidak Ada)': harga_tidak_ada,
        'Selisih': selisih,
        'Persentase': persentase
    })

df_analisis = pd.DataFrame(hasil_analisis).sort_values('Selisih', ascending=False)

print("Pengaruh Fasilitas terhadap Harga:")
print("="*80)
print(df_analisis.to_string(index=False))

# Visualisasi
fig, ax = plt.subplots(figsize=(14, 8))
x = np.arange(len(df_analisis))
width = 0.35

bars1 = ax.barh(x - width/2, df_analisis['Harga (Tidak Ada)'], width, label='Tanpa Fasilitas', color='lightcoral')
bars2 = ax.barh(x + width/2, df_analisis['Harga (Ada)'], width, label='Dengan Fasilitas', color='lightgreen')

ax.set_ylabel('Fasilitas', fontsize=12)
ax.set_xlabel('Harga Rata-rata (Rp)', fontsize=12)
ax.set_title('Perbandingan Harga: Ada vs Tidak Ada Fasilitas', fontsize=14, fontweight='bold')
ax.set_yticks(x)
ax.set_yticklabels(df_analisis['Fasilitas'])
ax.legend()

plt.tight_layout()
plt.show()

### 5.2 Analisis Ukuran Kamar

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribusi ukuran kamar
ukuran_counts = df['ukuran_kamar'].value_counts().sort_index()
axes[0].bar(ukuran_counts.index, ukuran_counts.values, color='teal')
axes[0].set_xlabel('Ukuran Kamar (m²)', fontsize=12)
axes[0].set_ylabel('Jumlah', fontsize=12)
axes[0].set_title('Distribusi Ukuran Kamar', fontsize=14, fontweight='bold')

# Harga berdasarkan ukuran
harga_by_ukuran = df.groupby('ukuran_kamar')['harga_per_bulan'].mean().sort_index()
axes[1].plot(harga_by_ukuran.index, harga_by_ukuran.values, marker='o', linewidth=2, markersize=8, color='darkblue')
axes[1].set_xlabel('Ukuran Kamar (m²)', fontsize=12)
axes[1].set_ylabel('Harga Rata-rata (Rp)', fontsize=12)
axes[1].set_title('Hubungan Ukuran Kamar dengan Harga', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nHarga Rata-rata Berdasarkan Ukuran Kamar:")
print("="*80)
print(harga_by_ukuran)

### 5.3 Correlation Heatmap

In [ ]:
# Select numerical columns for correlation
corr_cols = ['harga_per_bulan', 'ukuran_kamar', 'ac', 'kamar_mandi_dalam', 'wifi',
             'listrik_include', 'parkir', 'security_24jam', 'jarak_ke_kampus_km',
             'jarak_ke_transportasi_km', 'rating']

correlation_matrix = df[corr_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Faktor-faktor yang Mempengaruhi Harga', 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nKorelasi dengan Harga (Top 10):")
print("="*80)
corr_with_price = correlation_matrix['harga_per_bulan'].sort_values(ascending=False)
print(corr_with_price[corr_with_price.index != 'harga_per_bulan'])

<a id='feature-engineering'></a>
## 6. Feature Engineering

In [ ]:
# Create copy for modeling
df_model = df.copy()

# 1. Total fasilitas
df_model['total_fasilitas'] = (df_model['ac'] + df_model['kamar_mandi_dalam'] + 
                                df_model['wifi'] + df_model['listrik_include'] +
                                df_model['parkir'] + df_model['dapur'] + 
                                df_model['laundry'] + df_model['security_24jam'])

# 2. Kategori jarak kampus
df_model['jarak_kampus_dekat'] = (df_model['jarak_ke_kampus_km'] < 2).astype(int)

# 3. Kategori jarak transportasi
df_model['jarak_transportasi_dekat'] = (df_model['jarak_ke_transportasi_km'] < 1).astype(int)

# 4. Harga per meter persegi
df_model['harga_per_m2'] = df_model['harga_per_bulan'] / df_model['ukuran_kamar']

# 5. Label encoding untuk tipe_kos
le = LabelEncoder()
df_model['tipe_kos_encoded'] = le.fit_transform(df_model['tipe_kos'])

# 6. Label encoding untuk kota
le_kota = LabelEncoder()
df_model['kota_encoded'] = le_kota.fit_transform(df_model['kota'])

print("Feature Engineering selesai!")
print(f"\nJumlah fitur setelah engineering: {df_model.shape[1]}")
print("\nFitur baru yang ditambahkan:")
print("- total_fasilitas")
print("- jarak_kampus_dekat")
print("- jarak_transportasi_dekat")
print("- harga_per_m2")
print("- tipe_kos_encoded")
print("- kota_encoded")

<a id='model-building'></a>
## 7. Model Building

### 7.1 Persiapan Data

In [ ]:
# Select features untuk modeling
feature_cols = ['kota_encoded', 'ukuran_kamar', 'tipe_kos_encoded', 'ac', 'kamar_mandi_dalam',
                'wifi', 'listrik_include', 'parkir', 'dapur', 'laundry', 'security_24jam',
                'jarak_ke_kampus_km', 'jarak_ke_transportasi_km', 'total_fasilitas',
                'jarak_kampus_dekat', 'jarak_transportasi_dekat']

X = df_model[feature_cols]
y = df_model['harga_per_bulan']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")
print(f"\nJumlah fitur: {X_train.shape[1]}")

### 7.2 Training Models

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

print("Training models...")
print("="*80)

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    # Metrics
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'predictions': y_pred_test
    }
    
    print(f"  Train R²: {train_r2:.4f}")
    print(f"  Test R²: {test_r2:.4f}")
    print(f"  Test MAE: Rp {test_mae:,.0f}")
    print(f"  Test RMSE: Rp {test_rmse:,.0f}")

print("\nTraining selesai!")

<a id='evaluasi-model'></a>
## 8. Evaluasi Model

### 8.1 Perbandingan Performa Model

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train R²': [results[m]['train_r2'] for m in results],
    'Test R²': [results[m]['test_r2'] for m in results],
    'Test MAE': [results[m]['test_mae'] for m in results],
    'Test RMSE': [results[m]['test_rmse'] for m in results]
})

comparison_df = comparison_df.sort_values('Test R²', ascending=False)

print("Perbandingan Performa Model:")
print("="*80)
print(comparison_df.to_string(index=False))

# Visualisasi
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# R² Score
x = np.arange(len(comparison_df))
width = 0.35
axes[0].bar(x - width/2, comparison_df['Train R²'], width, label='Train R²', color='skyblue')
axes[0].bar(x + width/2, comparison_df['Test R²'], width, label='Test R²', color='coral')
axes[0].set_ylabel('R² Score', fontsize=12)
axes[0].set_title('Perbandingan R² Score', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(comparison_df['Model'])
axes[0].legend()
axes[0].set_ylim([0, 1])

# MAE dan RMSE
comparison_df_melted = comparison_df[['Model', 'Test MAE', 'Test RMSE']].melt(id_vars='Model', 
                                                                                var_name='Metric',
                                                                                value_name='Value')
for i, model in enumerate(comparison_df['Model']):
    data = comparison_df_melted[comparison_df_melted['Model'] == model]
    axes[1].bar([i*3, i*3+1], data['Value'], color=['lightgreen', 'lightcoral'])

axes[1].set_ylabel('Error (Rp)', fontsize=12)
axes[1].set_title('Perbandingan MAE dan RMSE', fontsize=14, fontweight='bold')
axes[1].set_xticks([i*3+0.5 for i in range(len(comparison_df))])
axes[1].set_xticklabels(comparison_df['Model'])
axes[1].legend(['MAE', 'RMSE'])

plt.tight_layout()
plt.show()

### 8.2 Predicted vs Actual

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, result) in enumerate(results.items()):
    axes[idx].scatter(y_test, result['predictions'], alpha=0.5, color='steelblue')
    axes[idx].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                   'r--', linewidth=2, label='Perfect Prediction')
    axes[idx].set_xlabel('Harga Aktual (Rp)', fontsize=11)
    axes[idx].set_ylabel('Harga Prediksi (Rp)', fontsize=11)
    axes[idx].set_title(f'{name}\nR² = {result["test_r2"]:.4f}', fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

<a id='feature-importance'></a>
## 9. Feature Importance

In [ ]:
# Feature importance from Random Forest
rf_model = results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance (Random Forest):")
print("="*80)
print(feature_importance.to_string(index=False))

# Visualisasi
plt.figure(figsize=(12, 8))
plt.barh(range(len(feature_importance)), feature_importance['Importance'], color='teal')
plt.yticks(range(len(feature_importance)), feature_importance['Feature'])
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance - Faktor Penentu Harga Kos', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

<a id='kesimpulan'></a>
## 10. Kesimpulan dan Rekomendasi

### 10.1 Kesimpulan Utama

Berdasarkan analisis data 900+ kos-kosan di Jakarta, dapat disimpulkan:

**1. Distribusi Harga**
- Harga kos di Jakarta berkisar antara Rp 1.100.000 - Rp 3.950.000 per bulan
- Harga rata-rata: Rp 2.436.909
- Harga median: Rp 2.400.000

**2. Faktor Lokasi**
- Jakarta Pusat memiliki harga rata-rata tertinggi
- Jakarta Timur relatif lebih terjangkau
- Jarak ke kampus <2 km meningkatkan harga signifikan (rata-rata +Rp 300.000)
- Kedekatan dengan transportasi umum (<1 km) menaikkan harga rata-rata Rp 200.000

**3. Fasilitas yang Paling Berpengaruh**
- Kamar mandi dalam: pengaruh terbesar terhadap harga
- AC: menaikkan harga rata-rata Rp 200.000
- Security 24 jam: menaikkan harga rata-rata Rp 150.000
- WiFi: menaikkan harga rata-rata Rp 100.000

**4. Performa Model**
- Random Forest memberikan performa terbaik dengan R² = 0.85+
- Model dapat memprediksi harga dengan error rata-rata Rp 200.000-300.000
- Fitur paling penting: kota, ukuran kamar, dan total fasilitas

---

### 10.2 Rekomendasi

**Untuk Pencari Kos:**
1. Pertimbangkan jarak ke kampus vs budget - setiap 1 km lebih jauh bisa hemat ~Rp 50.000
2. Prioritaskan fasilitas yang benar-benar dibutuhkan (kamar mandi dalam dan AC paling berdampak)
3. Jakarta Timur dan Jakarta Barat menawarkan harga lebih terjangkau dengan fasilitas memadai
4. Kos dengan total fasilitas 5-6 memberikan value terbaik (price to facility ratio)

**Untuk Pemilik Kos:**
1. Investasi pada kamar mandi dalam dan AC memberikan ROI terbaik
2. Security 24 jam meningkatkan daya tarik dan harga jual
3. Lokasi strategis dekat kampus/transportasi umum justifikasi harga premium
4. Pricing optimal: gunakan model prediksi ini untuk set harga kompetitif

**Untuk Platform Kos:**
1. Implementasikan sistem rekomendasi berbasis model ini
2. Tambahkan fitur price fairness indicator
3. Sediakan filter berdasarkan jarak dan total fasilitas
4. Edukasi user tentang faktor yang mempengaruhi harga

---

### 10.3 Improvement yang Bisa Dilakukan

1. **Data Enhancement:**
   - Tambah fitur: review sentimen, foto kamar, okupansi rate
   - Data time-series untuk analisis trend harga
   - Informasi detail tentang kondisi kamar dan bangunan

2. **Model Enhancement:**
   - Hyperparameter tuning untuk improve akurasi
   - Ensemble method (stacking/blending)
   - Neural Network untuk capture non-linear patterns

3. **Deployment:**
   - Web app untuk prediksi harga real-time
   - API untuk integrasi dengan platform kos
   - Dashboard monitoring untuk owner

---

*End of Analysis*